In [1]:
!git clone https://github.com/sirzechs66/custom_hunyuan-3d-2GP.git

Cloning into 'custom_hunyuan-3d-2GP'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 229 (delta 0), reused 3 (delta 0), pack-reused 224 (from 1)
Receiving objects: 100% (229/229), 74.41 MiB | 22.22 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [22]:
%cd /kaggle/working/custom_hunyuan-3d-2GP/Hunyuan3D-2GP
!pip install -r requirements.txt
%cd /kaggle/working/custom_hunyuan-3d-2GP/Hunyuan3D-2GP/hy3dgen/texgen/custom_rasterizer
!python setup.py install
%cd /kaggle/working/custom_hunyuan-3d-2GP/Hunyuan3D-2GP/hy3dgen/texgen/differentiable_renderer
!python setup.py install
%cd /kaggle/working/custom_hunyuan-3d-2GP/Hunyuan3D-2GP
!sed -i 's/    uvicorn.run(app, host=args.host, port=args.port, workers=1)/    demo.launch(share=True)/g' gradio_app.py
!pip install pydantic==2.10.6

/kaggle/working/custom_hunyuan-3d-2GP/Hunyuan3D-2GP
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/mesh_processor-0.0.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
/kaggle/working/custom_hunyuan-3d-2GP/Hunyuan3D-2GP/hy3dgen/texgen/custom_rasterizer
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self

In [21]:
%run gradio_app.py --enable_t23d --profile 5

Exception: File `'gradio_app.py'` not found.

## Access Generation Statistics


In [13]:
# Access the captured generation statistics
try:
    # Get the last generation
    last_stats = get_last_generation()
    
    # Get all generation history
    all_generations = get_generation_history()
    
    if last_stats:
        print("=== LAST GENERATION STATS ===")
        print(f"Total generations captured: {len(all_generations)}")
        print(f"Prompt: {last_stats['generation_info']['prompt']}")
        print(f"Folder: {last_stats['generation_info']['folder_name']}")
        print(f"Directory: {last_stats['generation_info']['folder_directory']}")
        
        if 'white_mesh' in last_stats['generation_info']:
            mesh_info = last_stats['generation_info']['white_mesh']
            print(f"White Mesh: {mesh_info['mesh_name']} ({mesh_info['mesh_format']})")
            
        if 'textured_mesh' in last_stats['generation_info']:
            mesh_info = last_stats['generation_info']['textured_mesh']
            print(f"Textured Mesh: {mesh_info['mesh_name']} ({mesh_info['mesh_format']})")
            
        print(f"Faces: {last_stats['number_of_faces']}")
        print(f"Vertices: {last_stats['number_of_vertices']}")
        
        # Display the complete stats structure
        print("\n=== COMPLETE STATS STRUCTURE ===")
        print(json.dumps(last_stats, indent=2, default=str))
        
    else:
        print("No generation data found. Please run the Gradio app and generate some 3D models first.")
        
except NameError:
    print("Generation functions not available. Please run the gradio_app.py first using %run.")

=== LAST GENERATION STATS ===
Total generations captured: 2
Prompt: 
Folder: 1559b62d-4fdf-4497-abe6-faf3f908fd61
Directory: gradio_cache/1559b62d-4fdf-4497-abe6-faf3f908fd61
White Mesh: white_mesh.glb (glb)
Faces: 538912
Vertices: 269449

=== COMPLETE STATS STRUCTURE ===
Generation functions not available. Please run the gradio_app.py first using %run.


## Extract Mesh Properties

Analyze the generated mesh statistics and file information:

In [14]:
# Extract mesh properties
if last_stats:
    print("=== MESH STATISTICS ===")
    num_faces = last_stats.get('number_of_faces', 0)
    num_vertices = last_stats.get('number_of_vertices', 0)
    
    print(f"Number of Faces: {num_faces:,}")
    print(f"Number of Vertices: {num_vertices:,}")
    
    # Calculate mesh complexity metrics
    if num_faces > 0 and num_vertices > 0:
        face_to_vertex_ratio = num_faces / num_vertices
        print(f"Face-to-Vertex Ratio: {face_to_vertex_ratio:.2f}")
        
        # Estimate mesh complexity
        if num_faces < 1000:
            complexity = "Low"
        elif num_faces < 10000:
            complexity = "Medium"
        elif num_faces < 50000:
            complexity = "High"
        else:
            complexity = "Very High"
        print(f"Mesh Complexity: {complexity}")
    
    # Extract mesh file information
    generation_info = last_stats.get('generation_info', {})
    
    if 'white_mesh' in generation_info:
        white_mesh = generation_info['white_mesh']
        print(f"\n=== WHITE MESH INFO ===")
        print(f"Filename: {white_mesh.get('mesh_name', 'N/A')}")
        print(f"Format: {white_mesh.get('mesh_format', 'N/A')}")
        print(f"Path: {white_mesh.get('mesh_path', 'N/A')}")
        print(f"File exists: {os.path.exists(white_mesh.get('mesh_path', ''))}")
        
        # Get file size if it exists
        mesh_path = white_mesh.get('mesh_path', '')
        if os.path.exists(mesh_path):
            file_size = os.path.getsize(mesh_path)
            print(f"File size: {file_size / (1024*1024):.2f} MB")
    
    if 'textured_mesh' in generation_info:
        textured_mesh = generation_info['textured_mesh']
        print(f"\n=== TEXTURED MESH INFO ===")
        print(f"Filename: {textured_mesh.get('mesh_name', 'N/A')}")
        print(f"Format: {textured_mesh.get('mesh_format', 'N/A')}")
        print(f"Path: {textured_mesh.get('mesh_path', 'N/A')}")
        print(f"File exists: {os.path.exists(textured_mesh.get('mesh_path', ''))}")
        
        # Get file size if it exists
        mesh_path = textured_mesh.get('mesh_path', '')
        if os.path.exists(mesh_path):
            file_size = os.path.getsize(mesh_path)
            print(f"File size: {file_size / (1024*1024):.2f} MB")
    
    # Store mesh info in global variables for easy access
    globals()['mesh_faces'] = num_faces
    globals()['mesh_vertices'] = num_vertices
    globals()['mesh_files'] = generation_info
    
    print(f"\n=== MESH VARIABLES FOR NOTEBOOK ACCESS ===")
    print(f"mesh_faces = {num_faces}")
    print(f"mesh_vertices = {num_vertices}")
    print("mesh_files = generation_info dictionary")
    
else:
    print("No mesh data available. Please generate a 3D model first.")

=== MESH STATISTICS ===
Number of Faces: 538,912
Number of Vertices: 269,449
Face-to-Vertex Ratio: 2.00
Mesh Complexity: Very High

=== WHITE MESH INFO ===
Filename: white_mesh.glb
Format: glb
Path: gradio_cache/1559b62d-4fdf-4497-abe6-faf3f908fd61/white_mesh.glb
File exists: True
File size: 9.25 MB

=== MESH VARIABLES FOR NOTEBOOK ACCESS ===
mesh_faces = 538912
mesh_vertices = 269449
mesh_files = generation_info dictionary


In [10]:
!pip install pyvista

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/mesh_processor-0.0.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached pyvista-0.45.3-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.7 MB/s eta 0:00:00a 0:00:01


In [11]:
import pyvista as pv 
import trimesh

exporting /tmp/gradio/9b9c4f854fd5e27f75f6c8022cad304ab92626ba26bb57a2538e7400e94d1444/white_mesh.glb
reduce face to 10000
Created new folder: gradio_cache/88dd3a48-58d6-44d4-9937-58f1247ca69a
Created new folder: gradio_cache/69fa619f-189d-40ae-bbcb-512480ee59b4
Find html file gradio_cache/69fa619f-189d-40ae-bbcb-512480ee59b4/white_mesh.html, True, relative HTML path is /static/69fa619f-189d-40ae-bbcb-512480ee59b4/white_mesh.html
export to gradio_cache/88dd3a48-58d6-44d4-9937-58f1247ca69a/white_mesh.glb
Mesh info: {'mesh_name': 'white_mesh.glb', 'mesh_format': 'glb', 'mesh_path': 'gradio_cache/88dd3a48-58d6-44d4-9937-58f1247ca69a/white_mesh.glb', 'textured': False}
Created new folder: gradio_cache/1559b62d-4fdf-4497-abe6-faf3f908fd61


Volume Decoding: 100%|██████████| 2122/2122 [00:59<00:00, 35.51it/s]
2025-07-29 21:10:43,930 - hy3dgen.shapgen - INFO - ---Shape generation takes 81.98956894874573 seconds ---


Find html file gradio_cache/1559b62d-4fdf-4497-abe6-faf3f908fd61/white_mesh.html, True, relative HTML path is /static/1559b62d-4fdf-4497-abe6-faf3f908fd61/white_mesh.html
=== GENERATION SUMMARY ===
Prompt: 
Folder Name: 1559b62d-4fdf-4497-abe6-faf3f908fd61
Folder Directory: gradio_cache/1559b62d-4fdf-4497-abe6-faf3f908fd61
White Mesh: white_mesh.glb (glb)


In [ ]:
# Advanced 3D Mesh Visualization and Analysis
try:
    if 'mesh_files' in globals() and mesh_files:
        print("=== 3D MESH VISUALIZATION ===")
        
        # Load and visualize the white mesh
        if 'white_mesh' in mesh_files:
            white_mesh_path = mesh_files['white_mesh']['mesh_path']
            if os.path.exists(white_mesh_path):
                print(f"Loading mesh: {white_mesh_path}")
                
                # Load with trimesh for analysis
                mesh = trimesh.load(white_mesh_path)
                
                print(f"Mesh loaded successfully!")
                print(f"Vertices: {len(mesh.vertices):,}")
                print(f"Faces: {len(mesh.faces):,}")
                print(f"Watertight: {mesh.is_watertight}")
                print(f"Volume: {mesh.volume:.4f} cubic units")
                print(f"Surface Area: {mesh.area:.4f} square units")
                print(f"Bounding Box: {mesh.bounds}")
                
                # Check mesh quality
                if hasattr(mesh, 'is_valid'):
                    print(f"Valid Mesh: {mesh.is_valid}")
                
                # Analyze mesh properties
                if hasattr(mesh, 'euler_number'):
                    print(f"Euler Number: {mesh.euler_number}")
                
                # Convert to PyVista for advanced visualization
                try:
                    # Convert trimesh to pyvista
                    faces_pv = mesh.faces.reshape(-1, 4)  # Assuming triangular faces
                    faces_pv[:, 0] = 3  # Set the first column to 3 (triangle)
                    
                    pv_mesh = pv.PolyData(mesh.vertices, faces_pv)
                    
                    print(f"\n=== PYVISTA MESH ANALYSIS ===")
                    print(f"Number of points: {pv_mesh.n_points}")
                    print(f"Number of cells: {pv_mesh.n_cells}")
                    print(f"Memory usage: {pv_mesh.memory_usage(deep=True) / 1024**2:.2f} MB")
                    
                    # Calculate mesh quality metrics
                    quality = pv_mesh.compute_cell_quality()
                    print(f"Average cell quality: {quality['CellQuality'].mean():.4f}")
                    print(f"Min cell quality: {quality['CellQuality'].min():.4f}")
                    print(f"Max cell quality: {quality['CellQuality'].max():.4f}")
                    
                    # Store for further analysis
                    globals()['current_mesh'] = mesh
                    globals()['current_pv_mesh'] = pv_mesh
                    
                    print(f"\n=== MESH VARIABLES AVAILABLE ===")
                    print("• current_mesh - Trimesh object for analysis")
                    print("• current_pv_mesh - PyVista mesh for visualization")
                    
                except Exception as e:
                    print(f"PyVista conversion error: {e}")
                    print("Trimesh analysis completed successfully")
                    globals()['current_mesh'] = mesh
                
            else:
                print(f"Mesh file not found: {white_mesh_path}")
        
        # Also analyze textured mesh if available
        if 'textured_mesh' in mesh_files:
            textured_mesh_path = mesh_files['textured_mesh']['mesh_path']
            if os.path.exists(textured_mesh_path):
                print(f"\n=== TEXTURED MESH ANALYSIS ===")
                textured_mesh = trimesh.load(textured_mesh_path)
                print(f"Textured mesh vertices: {len(textured_mesh.vertices):,}")
                print(f"Textured mesh faces: {len(textured_mesh.faces):,}")
                
                # Compare file sizes
                white_size = os.path.getsize(white_mesh_path) / (1024**2)
                textured_size = os.path.getsize(textured_mesh_path) / (1024**2)
                print(f"File size comparison:")
                print(f"  White mesh: {white_size:.2f} MB")
                print(f"  Textured mesh: {textured_size:.2f} MB")
                print(f"  Texture overhead: {((textured_size - white_size) / white_size * 100):.1f}%")
                
                globals()['current_textured_mesh'] = textured_mesh
        
    else:
        print("No mesh files available. Please generate a 3D model first.")
        
except Exception as e:
    print(f"Error during mesh analysis: {e}")
    print("Make sure you have generated a 3D model and mesh_files variable is available.")

In [ ]:
# Interactive Mesh Visualization with PyVista
try:
    if 'current_pv_mesh' in globals():
        print("=== INTERACTIVE 3D VISUALIZATION ===")
        
        # Create a plotter for interactive visualization
        plotter = pv.Plotter(notebook=True, window_size=[800, 600])
        
        # Add the mesh with nice lighting
        plotter.add_mesh(
            current_pv_mesh, 
            color='lightgray', 
            show_edges=True,
            edge_color='black',
            line_width=0.5,
            opacity=0.9
        )
        
        # Add lighting and camera settings
        plotter.add_light(pv.Light(position=(1, 1, 1)))
        plotter.camera_position = 'isometric'
        plotter.background_color = 'white'
        
        # Show the plot
        plotter.show()
        
        print("Interactive 3D mesh visualization displayed above.")
        print("You can rotate, zoom, and pan the mesh using mouse controls.")
        
    elif 'current_mesh' in globals():
        print("=== BASIC MESH VISUALIZATION ===")
        print("PyVista not available, using matplotlib for basic visualization...")
        
        # Create a basic matplotlib 3D plot
        fig = plt.figure(figsize=(12, 8))
        ax = fig.add_subplot(111, projection='3d')
        
        # Plot the mesh vertices
        vertices = current_mesh.vertices
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], 
                  c='blue', s=0.1, alpha=0.6)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('3D Mesh Point Cloud')
        
        plt.tight_layout()
        plt.show()
        
        print("Basic mesh visualization completed.")
        
    else:
        print("No mesh available for visualization.")
        print("Please run the mesh analysis cell first to load a mesh.")
        
except Exception as e:
    print(f"Visualization error: {e}")
    print("Ensure PyVista is installed: pip install pyvista")

In [ ]:
# Mesh Comparison and Export Utilities
def compare_generations(stats_list=None):
    """Compare multiple generation statistics"""
    if stats_list is None:
        stats_list = get_generation_history()
    
    if len(stats_list) < 2:
        print("Need at least 2 generations to compare.")
        return
    
    print("=== GENERATION COMPARISON ===")
    
    # Create comparison DataFrame
    comparison_data = []
    for i, stats in enumerate(stats_list[-5:]):  # Last 5 generations
        if 'mesh_info' in stats:
            mesh_info = stats['mesh_info']
            comparison_data.append({
                'Generation': i+1,
                'Prompt': stats.get('prompt', 'N/A')[:30] + '...' if len(stats.get('prompt', '')) > 30 else stats.get('prompt', 'N/A'),
                'Vertices': mesh_info.get('vertices', 0),
                'Faces': mesh_info.get('faces', 0),
                'File Size (MB)': mesh_info.get('file_size_mb', 0),
                'Generation Time': stats.get('total_time', 'N/A'),
                'Model Type': stats.get('model_name', 'N/A')
            })
    
    if comparison_data:
        df = pd.DataFrame(comparison_data)
        print(df.to_string(index=False))
        
        # Plotting comparisons
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Vertices comparison
        axes[0,0].bar(df['Generation'], df['Vertices'])
        axes[0,0].set_title('Vertices Count by Generation')
        axes[0,0].set_xlabel('Generation')
        axes[0,0].set_ylabel('Vertices')
        
        # Faces comparison
        axes[0,1].bar(df['Generation'], df['Faces'])
        axes[0,1].set_title('Faces Count by Generation')
        axes[0,1].set_xlabel('Generation')
        axes[0,1].set_ylabel('Faces')
        
        # File size comparison
        axes[1,0].bar(df['Generation'], df['File Size (MB)'])
        axes[1,0].set_title('File Size by Generation')
        axes[1,0].set_xlabel('Generation')
        axes[1,0].set_ylabel('Size (MB)')
        
        # Generation time (if available)
        if any(t != 'N/A' for t in df['Generation Time']):
            time_values = [float(t) if t != 'N/A' else 0 for t in df['Generation Time']]
            axes[1,1].bar(df['Generation'], time_values)
            axes[1,1].set_title('Generation Time by Generation')
            axes[1,1].set_xlabel('Generation')
            axes[1,1].set_ylabel('Time (seconds)')
        else:
            axes[1,1].text(0.5, 0.5, 'No timing data available', 
                          ha='center', va='center', transform=axes[1,1].transAxes)
            axes[1,1].set_title('Generation Time (No Data)')
        
        plt.tight_layout()
        plt.show()
        
        return df
    else:
        print("No mesh information found in generation history.")

def export_analysis_report(output_dir=None):
    """Export comprehensive analysis report"""
    if output_dir is None:
        output_dir = os.path.join(os.getcwd(), "analysis_reports")
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Get latest generation stats
    latest_stats = get_last_generation()
    if not latest_stats:
        print("No generation data available for report.")
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_file = os.path.join(output_dir, f"hunyuan3d_analysis_{timestamp}.txt")
    
    with open(report_file, 'w') as f:
        f.write("HUNYUAN3D GENERATION ANALYSIS REPORT\n")
        f.write("="*50 + "\n\n")
        f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        # Generation details
        f.write("GENERATION DETAILS:\n")
        f.write(f"Prompt: {latest_stats.get('prompt', 'N/A')}\n")
        f.write(f"Model: {latest_stats.get('model_name', 'N/A')}\n")
        f.write(f"Output Directory: {latest_stats.get('output_dir', 'N/A')}\n")
        f.write(f"Generation Time: {latest_stats.get('total_time', 'N/A')} seconds\n\n")
        
        # Mesh information
        if 'mesh_info' in latest_stats:
            mesh_info = latest_stats['mesh_info']
            f.write("MESH PROPERTIES:\n")
            f.write(f"Vertices: {mesh_info.get('vertices', 'N/A'):,}\n")
            f.write(f"Faces: {mesh_info.get('faces', 'N/A'):,}\n")
            f.write(f"File Size: {mesh_info.get('file_size_mb', 'N/A')} MB\n")
            f.write(f"Format: {mesh_info.get('format', 'N/A')}\n")
            
            if 'current_mesh' in globals():
                mesh = globals()['current_mesh']
                f.write(f"Volume: {mesh.volume:.4f} cubic units\n")
                f.write(f"Surface Area: {mesh.area:.4f} square units\n")
                f.write(f"Watertight: {mesh.is_watertight}\n")
        
        # Generation history summary
        history = get_generation_history()
        f.write(f"\nTOTAL GENERATIONS IN SESSION: {len(history)}\n")
        
        if len(history) > 1:
            f.write("\nRECENT GENERATIONS:\n")
            for i, stats in enumerate(history[-5:], 1):
                f.write(f"{i}. {stats.get('prompt', 'N/A')[:50]}...\n")
    
    print(f"Analysis report exported to: {report_file}")
    return report_file

# Run comparison if we have multiple generations
print("=== MESH ANALYSIS UTILITIES ===")
print("Available functions:")
print("• compare_generations() - Compare multiple generations")
print("• export_analysis_report() - Export detailed analysis report")
print()

# Auto-run comparison if we have multiple generations
try:
    history = get_generation_history()
    if len(history) >= 2:
        print("Multiple generations detected. Running comparison...")
        comparison_df = compare_generations()
    else:
        print(f"Currently have {len(history)} generation(s) in history.")
        print("Generate more models to enable comparison features.")
except:
    print("No generation history available yet.")

In [ ]:
# Testing and Usage Instructions
print("=== HUNYUAN3D ANALYSIS NOTEBOOK READY ===")
print()
print("🎯 QUICK START GUIDE:")
print("1. Run all cells above to set up the environment")
print("2. Go to the Gradio interface and generate a 3D model")
print("3. Come back and run the analysis cells to view results")
print()
print("📊 AVAILABLE ANALYSIS FEATURES:")
print("• Real-time generation statistics")
print("• 3D mesh visualization and quality analysis")
print("• Generation comparison and benchmarking")
print("• Export capabilities for reports and data")
print()
print("🔧 USEFUL COMMANDS:")
print("• get_last_generation() - Get latest generation stats")
print("• get_generation_history() - Get all generation history")
print("• compare_generations() - Compare multiple generations")
print("• export_analysis_report() - Export detailed report")
print()
print("📈 CURRENT SESSION STATUS:")

try:
    last_gen = get_last_generation()
    history = get_generation_history()
    
    if last_gen:
        print(f"✅ Last Generation: '{last_gen.get('prompt', 'N/A')[:40]}...'")
        print(f"📁 Output Directory: {last_gen.get('output_dir', 'N/A')}")
        
        if 'mesh_info' in last_gen:
            mesh_info = last_gen['mesh_info']
            print(f"🔺 Mesh Quality: {mesh_info.get('vertices', 0):,} vertices, {mesh_info.get('faces', 0):,} faces")
            print(f"💾 File Size: {mesh_info.get('file_size_mb', 0):.2f} MB")
    else:
        print("❌ No generations found in current session")
    
    print(f"📚 Total Generations: {len(history)}")
    
    # Check if mesh is loaded for visualization
    if 'current_mesh' in globals():
        print("✅ 3D Mesh loaded and ready for analysis")
        if 'current_pv_mesh' in globals():
            print("✅ PyVista mesh available for advanced visualization")
    else:
        print("⏳ No mesh loaded - generate a model to enable 3D analysis")
        
except Exception as e:
    print(f"❌ Error checking session status: {e}")

print()
print("🚀 Ready for 3D generation and analysis!")
print("=" * 50)